In [1]:
#1. Import necessary libraries
import glob
import pandas as pd
import xml.etree.ElementTree as ET #deal with XML format
import json
from datetime import datetime

In [2]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip

--2023-08-24 17:30:31--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.45.118.108
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.45.118.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4249 (4.1K) [application/zip]
Saving to: 'datasource.zip'

     0K ....                                                  100%  427M=0s

2023-08-24 17:30:32 (427 MB/s) - 'datasource.zip' saved [4249/4249]



In [3]:
tmpfile    = "dealership_temp.tmp"               # file used to store all extracted data
logfile    = "dealership_logfile.txt"            # all event logs will be stored in this file
targetfile = "dealership_transformed_data.csv"   # file where transformed data is stored

In [12]:
#2. Extract functions:
def extract_from_csv(input_file):
    df = pd.read_csv(input_file)
    return df

def extract_from_json(input_file):
    df = pd.read_json(input_file, lines = True)
    return df

def extract_from_xml(input_file):
    df = pd.DataFrame(columns = ["car_model","year_of_manufacture","price","fuel"])
    tree = ET.parse(input_file)
    root = tree.getroot()
    for row in root:
        car_model = row.find("car_model").text
        year_of_manufacture = row.find("year_of_manufacture").text
        price = float(row.find("price").text)
        fuel = row.find("fuel").text
        df = df.append({"car_model" : car_model ,"year_of_manufacture": year_of_manufacture,"price" : price,"fuel": fuel}, ignore_index = True)
    return df


In [13]:
#3. Synthesizing extract function:
def extract():
    extracted_data = pd.DataFrame(columns = ["car_model","year_of_manufacture","price","fuel"])
    for csvfile in glob.glob("*.csv"):
        extracted_data = extracted_data.append(extract_from_csv(csvfile), ignore_index = True)
    for jsonfile in glob.glob("*.json"):
        extracted_data = extracted_data.append(extract_from_json(jsonfile), ignore_index = True)
    for xmlfile in glob.glob("*.xml"):
        extracted_data = extracted_data.append(extract_from_xml(xmlfile), ignore_index = True)
    return extracted_data
    

In [6]:
#4. Transform function:
def transform (data):
    data['price'] = round(data.price, 2)
    return data

In [7]:
def load(targetfile, data_to_load):
    data_to_load.to_csv(targetfile)

In [15]:
extracted_data = extract()

C:\Users\vagy1255\AppData\Local\Temp\ipykernel_20968\4279640944.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_data = extracted_data.append(extract_from_csv(csvfile), ignore_index = True)
C:\Users\vagy1255\AppData\Local\Temp\ipykernel_20968\4279640944.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_data = extracted_data.append(extract_from_csv(csvfile), ignore_index = True)
C:\Users\vagy1255\AppData\Local\Temp\ipykernel_20968\4279640944.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_data = extracted_data.append(extract_from_csv(csvfile), ignore_index = True)
C:\Users\vagy1255\AppData\Local\Temp\ipykernel_20968\4279640944.py:7: FutureWarning: The frame.append method is deprecated a

In [16]:
transformed_data = transform(extracted_data)

In [17]:
load(targetfile, transformed_data)